In [40]:
# CMSC426 Project 3
# GROUP MEMBERS: Andrew Sumner (asumner1), Teimuraz Trapaidze (ttrapaid)
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import cv2
from scipy import ndimage
import math
import random

In [228]:
images = []
for filename in os.listdir("images/dolphin"):
    image = img.imread(os.path.join("images/dolphin",filename))
    images.append(image)

In [229]:
# supposedly we get descriptors for all images in a category, then 
# stack ? 
sift = cv2.SIFT_create()
kp, descriptors = sift.detectAndCompute(images[0], None)
print(descriptors.shape)

(337, 128)


In [230]:
for pt, desc in zip(kp, descriptors):
    (x,y) = pt.pt
    print((x,y), desc)

(15.510043144226074, 88.27215576171875) [ 69.   3.   4.  53.  21.   0.   0.  26.   4.   3.  45.  81.  21.   1.
   1.   2.  75.  15.  18.  23.   3.   0.   0.  21.  13.   2.   0.   0.
   0.   0.   0.   6.  36.   4.   6.  90.  28.   0.   1.  19.  22.   9.
   6.  71.  88.   3.   3.  11. 142.  58.   6.  21.  13.   0.   0.  13.
  81.  14.   0.   0.   0.   0.   0.  31.  94.  28.  49.  54.  16.   1.
   0.  22.  34.   2.   1.  24. 102.  23.  27.  22. 142.  13.   0.   3.
   8.   5.  16.  57. 142.   7.   0.   0.   0.   0.   0. 142.  94.  16.
   8.   2.  12.   3.   4. 119.   9.   0.   0.   1.  12.  15.  49.  35.
  76.   2.   0.   0.   0.   6.  37. 109. 142.   1.   0.   0.   0.   0.
   0. 142.]
(17.65346336364746, 71.48989868164062) [  1.   4.  24.  36.   2.   0.   0.   0.  20.   9.   2.   6.   1.   0.
   0.   2.  17.   1.   0.   0.   0.   0.   0.   6.   0.   0.   0.   0.
   0.   0.   0.   0.   2.   7.  14.  18.   7.   0.   0.   1. 104.  21.
   2.   3.   2.   0.   1.  12. 176.  12.   0.  21.  45.  

(279.3529052734375, 115.81716918945312) [ 19.  33.   1.   2.  39.  30.   0.   0.  66.  73.   0.   1.  39.  55.
   1.   1. 112. 124.  68.  25.   4.   1.   0.   0.  45.  29.  54.  84.
  39.   6.  34.  55.  53.  39.   0.   0.  23.  37.   2.   3.  40.  24.
   0.   1.  94.  89.   2.   3. 124.  94.  16.   1.  13.   9.   1.  23.
 124.  54.  26.   3.   0.   0.   8.  67.  59.   8.   0.   0.  24.  84.
   4.   6.  65.   7.   0.   0.  47. 124.  16.  15. 124.  26.   0.   0.
  12.  63.  12.  36. 124.  67.  20.  11.   0.   0.   1.  24.  27.  12.
   0.   0.  36.  66.   3.   4.  73.  34.   0.   0.  21.  58.   9.  11.
  50.  26.   0.   0.  33.  93.   7.   4.  98.  48.   3.   1.   0.   2.
   2.   7.]
(279.492431640625, 123.90182495117188) [ 67.  34.   8.  80.  52.   0.   0.   3.  85.  46.  10.  64.  53.   1.
   0.   2. 120.  30.   3.  22.  49.   4.   0.   5.  17.   3.   0.  15.
 120.   9.   0.   2. 113.   8.   1.  24.  90.   8.   0.  20. 105.  17.
   4.  60.  81.   2.   0.   8. 120.  42.   3.  20.  45.  

In [231]:
descriptors = np.vstack(descriptors)

In [232]:
print(descriptors.shape)
print(len(kp))

(337, 128)
337


In [233]:
# this k means clustering method is written for one image
# im not sure how, but were supposed to get it to work with 
# every image? 

# if you look at slide 43 of bovw, it says to set each cluster C_i 
# to be the set of points X that are closer to the center c_i than 
# any other center ... but what is X if were using every feature vector
# from every image? if its one image, X would just be the set of points 
# of each descriptor/keypoint for that particular image, but does it 
# work like that if youre using every image?

# if so, we would just have to pass in keypoints from every image 
# and descriptors from every image instead of just one like i did 
# and this method will probably work

# 428 on piazza seems to be the same question I have, but its unanswered 
# as of writing this

# using slide 43 on bagof visual words lecture
def clustering(keypoints, descriptors, k):
    num_descs = len(keypoints)
    
    centers = []
    center_descs = []
    # initial random k centers
    for i in range(k):
        center = random.randint(0, num_descs-1)
        centers.append((keypoints[center].pt[0], keypoints[center].pt[1]))
        center_descs.append(descriptors[center])
        
    print(len(centers))
    old_centers = [] 
    
    while (old_centers != centers):
        
        clusters = {center:[] for center in centers}
        print(len(clusters.items()))
        for keypoint, descriptor in zip(keypoints, descriptors):
            closest = 99999999
            for center, center_desc in zip(centers, center_descs):
                dist = np.sum((center_desc-descriptor)**2)
                if dist < closest:
                    closest = dist
                    closest_center = center
            clusters[closest_center].append(np.array([keypoint.pt[0], keypoint.pt[1]]))
        
        old_centers = centers
        centers = []
        
        
        for center, cluster in clusters.items():
            if (len(cluster) > 0):
                cluster = np.vstack(cluster)
                new_center = np.sum(cluster,axis=0)/np.linalg.norm(cluster)
                centers.append((new_center[0], new_center[1]))

            
    return centers

In [234]:
# i havent checked this yet, but if you were to sum the number of key 
# points in each cluster, it should equal the number of descriptors 

# this is for one image

# issue: why is the number of clusters decreasing with each iteration

clustering(kp, descriptors, 200)

200
145
145
121
108
95
84
77
71
67
64
61
59
57
55
53
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27


[(1.5006401213305114, 0.7984452053507883),
 (2.86200970395683, 1.5050110627692914),
 (2.223620611232056, 0.9341293072064999),
 (4.20112734060255, 2.711706126351712),
 (1.3634833545356608, 1.703038887377863),
 (3.0267613563379605, 1.5471710523023492),
 (0.9304038071367486, 1.4601629304108814),
 (1.8767253757337643, 0.9135466107012341),
 (2.4958403142319985, 1.4900584863109043),
 (4.010638898275913, 2.0158630880427695),
 (1.4391048148468546, 1.2038964932458054),
 (2.5458391110244705, 1.5857573148061321),
 (1.9987355793118071, 0.9494408023174088),
 (3.957426731395741, 2.289881717314031),
 (2.965995438984879, 1.663199031363454),
 (2.1788440937266595, 0.9875253846177174),
 (2.3584561840485314, 1.2493089798206174),
 (5.104359089880773, 2.626162361141476),
 (1.3399460662529674, 0.4065971406568998),
 (2.9069629451962573, 1.681838526554207),
 (3.7234134254591047, 1.9791250869229264),
 (3.0197544987505647, 1.504832029601408),
 (0.8463501640364254, 1.1050381875789876),
 (3.243378672768056, 1.9537

In [224]:
# all images

all_keypoints = []
all_descriptors = np.array([])

for image in images:
    a, b = sift.detectAndCompute(image, None)
    all_keypoints = all_keypoints + a
    
    break

In [225]:
print(all_descriptors)
print(len(all_keypoints))

[]
337
